In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [2]:
file_url = "./make_file/(통합)_서울시_도로_리스트.csv"
road_url = "./data/서울시 도로노선 정보.csv"


In [4]:
# data = pd.read_csv(file_url, encoding="UTF-8", low_memory=False)
road_data = pd.read_csv(road_url, encoding="EUC-KR", low_memory=False)

In [4]:
data.head(2)

,도로명,열선,도로 종류,시작점_위도,시작점_경도,종료점_위도,종료점_경도,중앙점_위도,중앙점_경도,도로_길이,행정동
0,보광로12길,0,주거지 도로,37.526615,127.000794,37.526707,127.001085,37.526661,127.000940,27.605333,보광동
1,보광로12길,0,주거지 도로,37.526615,127.000794,37.526500,127.000620,37.526558,127.000707,20.002271,보광동


In [5]:
data['도로명'].value_counts()

도로명
한천로                         234
통일로                         233
통일로 중앙버스전용차로                213
남부순환로                       204
오패산로                        200
                           ... 
['선유로43나길', '선유로43가길']        1
['갈현로43길', '갈현로41길']          1
['선유로43길', '양평로21길']          1
['성내천로33가길', '성내천로29다길']      1
['독산로29길', '독산로25길']          1
Name: count, Length: 12513, dtype: int64

In [7]:
road_data.head()

,순번,노선명(도로명),도로종류,도로기능,도로규모,도로폭
0,1,가락로,일반도로,국지도로,소로3류,폭6-8m
1,2,가락로11길,일반도로,국지도로,소로,6m미만
2,3,가락로12길,일반도로,국지도로,소로2류,폭8-10m
3,4,가락로13길,일반도로,국지도로,소로,6m미만
4,5,가락로15길,일반도로,국지도로,소로,6m미만


In [13]:
road_data['도로폭'].value_counts()

도로폭
6m미만       21454
폭6-8m       3222
폭8-10m      1223
폭20-25m      599
폭10-12m      598
폭15-20m      479
폭25-30m      108
폭30-35m       49
폭35-40m       22
폭40-50m       14
폭50-70m        4
폭12-15m        3
Name: count, dtype: int64

In [7]:
road_data.rename(columns={'노선명(도로명)': '도로명'}, inplace=True)

In [8]:
# '도로기능'이 "도시고속도로"이거나 '도로종류'가 "자동차전용도로"인 행 삭제
road_data = road_data[~((road_data['도로기능'] == '도시고속도로') | (road_data['도로종류'] == '자동차전용도로'))]

In [9]:
road_data['도로폭'].value_counts()

도로폭
6m미만       21454
폭6-8m       3222
폭8-10m      1221
폭10-12m      598
폭20-25m      598
폭15-20m      478
폭25-30m      104
폭30-35m       49
폭35-40m       22
폭40-50m       12
폭50-70m        4
폭12-15m        3
Name: count, dtype: int64

In [10]:
road_data['도로규모'].value_counts()

도로규모
소로      21454
소로3류     3222
소로2류     1221
소로1류      598
중로1류      598
중로2류      478
확인불가      379
대로3류      104
대로2류       49
대로1류       22
광로3류       12
광로2류        4
중로3류        3
Name: count, dtype: int64

In [11]:
# 도로폭 구간과 평균값 매핑
mapping = {
    '6m미만': 3,
    '폭6-8m': 7,
    '폭8-10m': 9,
    '폭10-12m': 11,
    '폭12-15m': 13.5,
    '폭15-20m': 17.5,
    '폭20-25m': 22.5,
    '폭25-30m': 27.5,
    '폭30-35m': 32.5,
    '폭35-40m': 37.5,
    '폭40-50m': 45,
    '폭50-70m': 60
}

# 도로폭 컬럼을 평균값으로 치환
# road_data['도로폭'] = road_data['도로폭'].map(mapping)

In [12]:
# 도로규모 매핑 딕셔너리 정의
도로규모_mapping = {
    '소로': 1,
    '소로3류': 2,
    '소로2류': 3,
    '중로1류': 4,
    '소로1류': 5,
    '중로2류': 6,
    '대로3류': 7,
    '대로2류': 8,
    '대로1류': 9,
    '광로3류': 10,
    '광로2류': 11,
    '중로3류': 12,
    '확인불가': None  # '확인불가'는 None으로 처리
}

# 도로규모 컬럼을 매핑 적용
# road_data['도로규모'] = road_data['도로규모'].map(도로규모_mapping)
road_data['도로규모'] = road_data['도로규모'].replace('확인불가', None)

In [13]:
road_data.columns

Index(['순번', '도로명', '도로종류', '도로기능', '도로규모', '도로폭'], dtype='object')

In [14]:
print(f"{road_data['도로명'].isnull().sum()}")
print(f"{road_data['도로규모'].isnull().sum()}")
print(f"{road_data['도로폭'].isnull().sum()}") 

road_data = road_data.dropna(subset=['도로규모', '도로폭'], how='all')

print(f"{road_data['도로명'].isnull().sum()}")
print(f"{road_data['도로규모'].isnull().sum()}")
print(f"{road_data['도로폭'].isnull().sum()}") 

0
379
379
0
0
0


In [15]:
# 기존의 road_data를 복사하여 처리
new_rows = []

# road_data의 모든 행에 대해 반복
for _, row in road_data.iterrows():
    # '도로명'을 '~'로 분리하여 앞과 뒤로 나눈 후 새로운 행을 추가
    if '~' in row['도로명']:  # '~'가 있는 경우만 처리
        도로명_앞, 도로명_뒤 = row['도로명'].split('~')
        
        # 앞 도로명 추가
        new_rows.append({
            '순번': row['순번'],
            '도로명': 도로명_앞,
            '도로종류': row['도로종류'],
            '도로기능': row['도로기능'],
            '도로규모': row['도로규모'],
            '도로폭': row['도로폭']
        })
        
        # 뒤 도로명 추가
        new_rows.append({
            '순번': row['순번'],
            '도로명': 도로명_뒤,
            '도로종류': row['도로종류'],
            '도로기능': row['도로기능'],
            '도로규모': row['도로규모'],
            '도로폭': row['도로폭']
        })
    else:
        # '~'가 없는 경우 기존 데이터를 그대로 추가
        new_rows.append({
            '순번': row['순번'],
            '도로명': row['도로명'],
            '도로종류': row['도로종류'],
            '도로기능': row['도로기능'],
            '도로규모': row['도로규모'],
            '도로폭': row['도로폭']
        })

# 새로운 데이터프레임으로 변환하여 기존 road_data에 덮어쓰기
road_data = pd.DataFrame(new_rows)


In [16]:
road_data = road_data.drop_duplicates(subset='도로명', keep='first')

# 중복된 '도로명' 출력
duplicates = road_data[road_data.duplicated(subset='도로명', keep=False)]

# 중복된 '도로명' 출력
for idx, row in duplicates.iterrows():
    print(f"{row}")

In [17]:
# '도로폭'과 '도로규모' 데이터를 '도로명'을 기준으로 매핑
data['도로폭'] = data['도로명'].map(road_data.set_index('도로명')['도로폭'])
data['도로규모'] = data['도로명'].map(road_data.set_index('도로명')['도로규모'])

In [18]:
data['도로폭'].value_counts()

도로폭
6m미만       112049
폭6-8m       28136
폭8-10m       9170
폭10-12m      3754
폭20-25m      3076
폭15-20m      2709
폭25-30m       390
폭30-35m        93
폭35-40m         2
Name: count, dtype: int64

In [19]:
data['도로규모'].value_counts()

도로규모
소로      112049
소로3류     28136
소로2류      9170
소로1류      3754
중로1류      3076
중로2류      2709
대로3류       390
대로2류        93
대로1류         2
Name: count, dtype: int64

In [20]:
data.isnull().sum()

도로명          0
열선           0
도로 종류     1310
시작점_위도       0
시작점_경도       0
종료점_위도       0
종료점_경도       0
중앙점_위도       0
중앙점_경도       0
도로_길이        0
행정동          0
도로폭       1983
도로규모      1983
dtype: int64

In [21]:
data['행정동'].value_counts()

행정동
미아동       4942
수유동       4679
신림동       3845
화곡동       3148
상계동       2762
          ... 
남대문로3가       1
장암동          1
향동동          1
회현동          1
훈정동          1
Name: count, Length: 463, dtype: int64

In [22]:
data

,도로명,열선,도로 종류,시작점_위도,시작점_경도,종료점_위도,종료점_경도,중앙점_위도,중앙점_경도,도로_길이,행정동,도로폭,도로규모
0,보광로12길,0,주거지 도로,37.526615,127.000794,37.526707,127.001085,37.526661,127.000940,27.605333,보광동,6m미만,소로
1,보광로12길,0,주거지 도로,37.526615,127.000794,37.526500,127.000620,37.526558,127.000707,20.002271,보광동,6m미만,소로
2,보광로14길,0,주거지 도로,37.526615,127.000794,37.527184,127.000468,37.526900,127.000631,69.877541,보광동,6m미만,소로
3,보광로7길,0,주거지 도로,37.525964,126.997422,37.525748,126.998131,37.525856,126.997777,66.892380,보광동,6m미만,소로
4,장문로,0,3차로,37.525964,126.997422,37.525966,126.997165,37.525965,126.997294,22.682960,보광동,6m미만,소로
...,...,...,...,...,...,...,...,...,...,...,...,...,...
161357,천호대로125길,0,주거지 도로,37.554907,127.087447,37.554335,127.087071,37.554621,127.087259,71.710310,중곡동,6m미만,소로
161358,천호대로119길,0,주거지 도로,37.556081,127.086125,37.556026,127.086131,37.556054,127.086128,6.118745,중곡동,6m미만,소로
161359,용마산로3길,0,주거지 도로,37.556081,127.086125,37.556115,127.086474,37.556098,127.086299,31.035093,중곡동,6m미만,소로
161360,천호대로119길,0,주거지 도로,37.556081,127.086125,37.557508,127.085895,37.556795,127.086010,159.957599,중곡동,6m미만,소로


In [23]:
print(f"저장하는 도로의 수 : {len(data):,}")

저장하는 도로의 수 : 161,362


In [24]:
try :
    save_url = "./make_file/(도로폭)_서울시_도로_목록.csv"
    data.to_csv(save_url, encoding="UTF-8", index=False)
    print(f"{save_url} 파일 저장 완료.")
except OSError as e:
    print(e)

./make_file/(도로폭)_서울시_도로_목록.csv 파일 저장 완료.
